In [65]:
import cv2
import glob
import os
import numpy as np
import nd2reader
import matplotlib.pyplot as plt
from PIL import Image
from nd2reader import ND2Reader
from collections import defaultdict


Loading paths

In [ ]:
paths = glob.glob(os.path.join("C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104", "**","*.tif"),recursive=True)
print(paths)

['C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\1\\0\\10x_1.tif', 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\1\\0\\EPIFITC_1.tif', 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\1\\0\\EPIRedTracker_1.tif', 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\2\\0\\10x_0.tif', 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\2\\0\\10x_2.tif', 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\2\\0\\EPIFITC_0.tif', 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\2\\0\\EPIFITC_1.tif', 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\2\\0\\EPIFITC_2.tif', 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/241104\\2\\0\\EPIRedTracker_0.tif', 'C:/Users/srboval1/OneDrive

In [139]:
def saving (img, field, i_path):
    img_res = cv2.resize(img, (800, 600))
    base_name = os.path.splitext(os.path.basename(i_path))[0]
    parts = base_name.split('_')

    output_subdir = os.path.join('C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/Viability/', f"{parts[0]}")
    os.makedirs(output_subdir, exist_ok=True)

 #   saving(img, i_path)      

Fluorescent images - no z-projection

In [246]:
def color_channel(channel):
    channel_index = None  
    if channel == 'EPIRedTracker':
        channel_index = 0
    elif channel == 'EPIFITC':
        channel_index = 1
    elif channel == '10x':
        channel_index = 2
    
    return channel_index

In [ ]:
image_data_list = []

for count, i in enumerate(paths):
    image = cv2.imread(i) #reads the image into a NumPy array.
    parts = os.path.splitext(os.path.basename(i))[0].split('_')
    channel = parts[0]
    partss = (i.split("/")[7]).split("\\")
    holder = (partss[1])
    gradient = (partss[2])

    channel_index = color_channel(channel)
    running_id = parts[1]
        
    image_info = {
        'holder': holder,
        'gradient': gradient,
        'running_id': running_id,  
        'channel_index': channel_index,
        'image': image  # Optionally, store the image array
    }
    image_data_list.append(image_info)

grouped_data = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))

In [249]:
for item in image_data_list:
    holder = item['holder']
    gradient = item['gradient']
    running_id = item['running_id']
    channel_index = item['channel_index']
    image = item['image']
    
    # Create a dictionary for each item
    image_info = {
        'holder': holder,
        'gradient': gradient,
        'running_id': running_id,
        'channel_index': channel_index,
        'image': image
    }
    
    # Append the item to the appropriate group
grouped_data[holder][gradient][running_id][channel_index].append({
        'image': image
    })
# Convert to a regular dictionary for easier inspection (optional)
grouped_data = dict(grouped_data)


In [256]:
import numpy as np
import cv2
from collections import defaultdict

# Grouping data structure
grouped_data = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))

# Loop through each item in the list
for item in image_data_list:
    holder = item['holder']
    gradient = item['gradient']
    running_id = item['running_id']
    channel_index = item['channel_index']
    image = item['image']

    # Append the item to the appropriate group
    grouped_data[holder][gradient][running_id][channel_index].append({
        'image': image
    })

# Convert to a regular dictionary for easier inspection (optional)
grouped_data = dict(grouped_data)

def create_overlay(channels, target_size=(2304, 2304)):
    num_channels = len(channels)
    weights = 1.0 / num_channels
    
    overlay = None
    x=0
    for channel_index, images in channels.items():
        for image_dict in images:
            image = image_dict['image']
            image_resized = cv2.resize(image, target_size)  # Resize to target size

            # Initialize the overlay with the target size if it's the first image
            if overlay is None:
                overlay = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)  # Initialize overlay as a 3-channel image

            if channel_index == '10x':  # Handle brightfield separately
                # Normalize and convert the brightfield image to a color representation
                brightfield_image_colored = cv2.applyColorMap(image, cv2.COLORMAP_JET)  # Using a colormap for visibility
                overlay = cv2.addWeighted(overlay, 1, brightfield_image_colored, 0.5, 0) 
            #Colorizing image
            else:
                channel_image = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8) #empty image with 3 channels, all pixels are 0 and so it is black
                channel_image[:, :, channel_index] = image_resized[:, :, channel_index]  # Place grayscale image into the specified channel
            #channel_image no has color information in one of its RGB channels
            
                overlay = cv2.addWeighted(overlay, 1, channel_image, 1, 0)
            x+=1
    return overlay

# Display the grouped data
for holder, gradients in grouped_data.items():
    print(f"Holder: {holder}")
    for gradient, running_ids in gradients.items():
        print(f"  Gradient: {gradient}")
        for running_id, channels in running_ids.items():
            print(f"  Running_id: {running_id}")
            
            overlay = create_overlay(channels)  # Extract images for overlay
            
            if overlay is not None:
                resized_overlay = cv2.resize(overlay, (600, 700))  # Resize for display
                cv2.imshow(f'Overlay for {holder} - {gradient} - Running ID {running_id}', resized_overlay)
                cv2.waitKey(0)  # Wait for a key press
                cv2.destroyAllWindows()  # Close the current window


Holder: 1
  Gradient: 0
  Running_id: 1
Holder: 2
  Gradient: 0
  Running_id: 0
  Running_id: 2
  Running_id: 1
  Gradient: 1
  Running_id: 0
  Running_id: 1


In [ ]:
grouped_data = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))

# Loop through each item in the list
for item in image_data_list:
    holder = item['holder']
    gradient = item['gradient']
    running_id = item['running_id']
    channel_index = item['channel_index']
    image = item['image']

    # Append the item to the appropriate group
    grouped_data[holder][gradient][running_id][channel_index].append({
        'image': image
    })

# Convert to a regular dictionary for easier inspection (optional)
grouped_data = dict(grouped_data)

def create_overlay(channels, target_size=(2304, 2304)):
    num_channels = len(channels)
    weights = 1.0 / num_channels
    
    overlay= None
    
    x = 0
    for channel_index, images in channels.items():
        for image_dict in images:
            image= image_dict['image']
            image_resized = cv2.resize(image, target_size)

            if x == 0:
                height, width = image.shape[:2]
            
            if overlay is None:
                overlay = np.zeros((height, width, 3), dtype=np.uint8)
            if channel_index == '10x':  # Handle brightfield separately
                # Normalize and convert the brightfield image to a color representation
                brightfield_image_colored = cv2.applyColorMap(image, cv2.COLORMAP_JET)  # Using a colormap for visibility
                overlay = cv2.addWeighted(overlay, 1, brightfield_image_colored, 0.5, 0) 
            #Colorizing image
            else:
                channel_image = np.zeros((height, width, 3), dtype=np.uint8) #empty image with 3 channels, all pixels are 0 and so it is black
                channel_image[:, :, channel_index] = image_resized[:, :, channel_index]  # Place grayscale image into the specified channel
            #channel_image no has color information in one of its RGB channels
            
                overlay = cv2.addWeighted(overlay, 1, channel_image, 1, 0)
            x+=1
    return overlay

# Display the grouped data
for holder, gradients in grouped_data.items():
    print(f"Holder: {holder}")
    for gradient, running_ids in gradients.items():
        print(f"  Gradient: {gradient}")
        for running_id, channels in running_ids.items():
            print(f"  Running_id: {running_id}")
            
            overlay = create_overlay(channels)  # Extract images for overlay
            
            if overlay is not None:
                resized_overlay = cv2.resize(overlay, (600, 700))

                cv2.imshow(f'Overlay for {holder} - {gradient} - Running ID {running_id}', resized_overlay)
                cv2.waitKey(0)  # Wait for a key press
                cv2.destroyAllWindows()  # Close the current window


Holder: 1
  Gradient: 0
  Running_id: 1
Holder: 2
  Gradient: 0
  Running_id: 0
  Running_id: 2
  Running_id: 1
  Gradient: 1
  Running_id: 0
  Running_id: 1


Fluorescent images with max z-projection